In [1]:
# import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from tensorflow.keras.metrics import Accuracy, Precision, Recall
from selenium import webdriver
import urllib.request
import zipfile
import random


In [2]:
# set up the image parameters
img_rows, img_cols = 150, 150
num_classes = 2
batch_size = 32

# DONOT CLOSE THE SELENEIUM AUTOMATED CHROME BROWSER
# Set the path to the chromedriver executable
# Replace with the path to your own chromedriver executable
chrome_path = 'path/to/chromedriver.exe'
driver = webdriver.Chrome(chrome_path)

# Navigate to the download page
driver.get('https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip')


local_zip = '/tmp/cats-and-dogs.zip'
urllib.request.urlretrieve('https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip', local_zip)

/var/folders/r0/hg5c83bj3ml4mrvpmnch9nyw0000gn/T/ipykernel_9574/779036462.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_path)


('/tmp/cats-and-dogs.zip', <http.client.HTTPMessage at 0x2959b9240>)

In [3]:
# Extract the zip file
zip_ref = zipfile.ZipFile(local_zip, 'r')

extract_path = os.getcwd() + '/PetImages'
zip_ref.extractall(extract_path)
# zip_ref.extractall('/PetImages')
zip_ref.close()
# Close the browser
driver.quit()

In [6]:
dog_dir = extract_path + '/PetImages/Dog'
cat_dir = extract_path + '/PetImages/Cat'

In [7]:
# read the dog images
dog_images = []
count = 0
for i in os.listdir(dog_dir):
    img = cv2.imread(os.path.join(dog_dir, i), cv2.IMREAD_GRAYSCALE)
    if img is not None:
        dog_images.append(img)
        count += 1;
        if count >1000:
            break;

# read the cat images
cat_images = []
count = 0;
for i in os.listdir(cat_dir):
    img = cv2.imread(os.path.join(cat_dir, i), cv2.IMREAD_GRAYSCALE)
    if img is not None:
        cat_images.append(img)
        count+=1;
        if count>1000:
            break;

In [8]:


# resize the images to a uniform size
dog_images = [cv2.resize(i, (img_rows, img_cols)) for i in dog_images]
cat_images = [cv2.resize(i, (img_rows, img_cols)) for i in cat_images]

# convert the images to arrays
dog_images = np.array(dog_images)
cat_images = np.array(cat_images)

# concatenate the two sets of images and create labels
X = np.concatenate((dog_images, cat_images), axis=0)
y = np.concatenate((np.zeros(len(dog_images)), np.ones(len(cat_images))), axis=0)

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# preprocess the data by normalizing and one-hot encoding the labels
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

# create the deep learning model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy', tf.keras.metrics.Precision() ])

# data augmentation
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')



Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [9]:
# fit the model on the augmented data
datagen.fit(X_train)
history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
steps_per_epoch=len(X_train) // batch_size,
epochs=3,
validation_data=(X_test, y_test),
validation_steps=len(X_test) // batch_size)

# evaluate the model on the test set
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test Precision:', score[2])

Epoch 1/3


/var/folders/r0/hg5c83bj3ml4mrvpmnch9nyw0000gn/T/ipykernel_9574/1983063589.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=batch_size),
2023-05-14 23:26:42.337782: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


50/50 [==============================] - 11s 194ms/step - loss: 2.1388 - accuracy: 0.5067 - precision: 0.5067 - val_loss: 0.6873 - val_accuracy: 0.5736 - val_precision: 0.5736
Epoch 2/3
50/50 [==============================] - 9s 188ms/step - loss: 0.6938 - accuracy: 0.5392 - precision: 0.5392 - val_loss: 0.6858 - val_accuracy: 0.5786 - val_precision: 0.5786
Epoch 3/3
50/50 [==============================] - 10s 192ms/step - loss: 0.6923 - accuracy: 0.5386 - precision: 0.5386 - val_loss: 0.6883 - val_accuracy: 0.5486 - val_precision: 0.5486
Test loss: 0.6882746815681458
Test accuracy: 0.5486284494400024
Test Precision: 0.5486284494400024


In [10]:
# compare with traditional machine learning models
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

In [11]:
y_train = y_train.argmax(axis=1)
y_test = y_test.argmax(axis=1)

In [12]:
# create the models
dt_clf = DecisionTreeClassifier()
knn_clf = KNeighborsClassifier()
svm_clf = SVC()



# fit the models on the training data
dt_clf.fit(X_train.reshape(X_train.shape[0], img_rows*img_cols), y_train)
knn_clf.fit(X_train.reshape(X_train.shape[0], img_rows*img_cols), y_train)
svm_clf.fit(X_train.reshape(X_train.shape[0], img_rows*img_cols), y_train)

# evaluate the models on the test data
dt_preds = dt_clf.predict(X_test.reshape(X_test.shape[0], img_rows*img_cols))
knn_preds = knn_clf.predict(X_test.reshape(X_test.shape[0], img_rows*img_cols))
svm_preds = svm_clf.predict(X_test.reshape(X_test.shape[0], img_rows*img_cols))

dt_accuracy = accuracy_score(y_test, dt_preds)
knn_accuracy = accuracy_score(y_test, knn_preds)
svm_accuracy = accuracy_score(y_test, svm_preds)

dt_loss = dt_clf.score(X_test.reshape(X_test.shape[0], img_rows*img_cols), y_test)
knn_loss = knn_clf.score(X_test.reshape(X_test.shape[0], img_rows*img_cols), y_test)
svm_loss = svm_clf.score(X_test.reshape(X_test.shape[0], img_rows*img_cols), y_test)

dt_precision = precision_score(y_test, dt_preds, average='weighted')
knn_precision = precision_score(y_test, knn_preds, average='weighted')
svm_precision = precision_score(y_test, svm_preds, average='weighted')



# print the results
print(' Comparing the performance of deep learning model with three traditional-[ Decion Tree, KNN, SVM ] machine learning models')


print(f'The accuracy of DeepLearning: {score[1]} vs Decision Tree: {dt_accuracy} vs KNN: {knn_accuracy} vs SVM: {svm_accuracy}')
print(f'The Loss of DeepLearning: {score[0]} vs Decision Tree: {dt_loss} vs KNN: {knn_loss} vs SVM: {svm_loss}')
print(f'The Precision of DeepLearning: {score[2]} vs Decision Tree: {dt_precision} vs KNN: {knn_precision} vs SVM: {svm_precision}')


 Comparing the performance of deep learning model with three traditional-[ Decion Tree, KNN, SVM ] machine learning models
The accuracy of DeepLearning: 0.5486284494400024 vs Decision Tree: 0.48877805486284287 vs KNN: 0.4763092269326683 vs SVM: 0.5985037406483791
The Loss of DeepLearning: 0.6882746815681458 vs Decision Tree: 0.48877805486284287 vs KNN: 0.4763092269326683 vs SVM: 0.5985037406483791
The Precision of DeepLearning: 0.5486284494400024 vs Decision Tree: 0.48895450230071796 vs KNN: 0.4758655178657804 vs SVM: 0.5985675621373108
